In [1]:
from rouge import Rouge
import pandas as pd
import json
import numpy as np
from nltk import word_tokenize
from tqdm import tqdm

In [2]:
df_first = pd.read_csv('1var_gpt_reviews_results.tsv', sep='\t')

In [3]:
df_first = df_first[(df_first['Generated_reviews'].isna() == False)&(df_first['Generated_reviews'] != '.')]

In [4]:
hyps = df_first['Generated_reviews'].tolist()
refs = df_first['true_end'].tolist()

In [5]:
rouge = Rouge()
scores = rouge.get_scores(hyps, refs, avg=True)

ROUGE-N – measures unigram, bigram, trigram and higher order n-gram overlap

ROUGE-L – measures longest matching sequence of words using LCS. An advantage of using LCS is that
it does not require consecutive matches but in-sequence matches that reflect sentence level word
order. Since it automatically includes longest in-sequence common n-grams, you don’t need a
predefined n-gram length.

In [10]:
pd.DataFrame.from_dict(scores).round(3)

,rouge-1,rouge-2,rouge-l
r,0.130,0.014,0.120
p,0.163,0.014,0.154
f,0.135,0.014,0.126


In [11]:
df = pd.read_csv('normal_res_with_generated_and_gen_end.tsv', sep='\t')

In [14]:
hyps = df['generated_end'].tolist()
refs = df['true_end'].tolist()

In [15]:
rouge = Rouge()
scores = rouge.get_scores(hyps, refs, avg=True)

In [17]:
pd.DataFrame.from_dict(scores).round(3)

,rouge-1,rouge-2,rouge-l
r,0.244,0.011,0.220
p,0.042,0.003,0.038
f,0.055,0.004,0.049


### Включенность ключевых слов

In [151]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'index'], axis=1)
df = df.drop_duplicates('id').reset_index(drop=True)

In [152]:
with open('baseline_reviews.json', 'r', encoding='utf-8') as f:
    baseline_reviews = json.loads(f.read())

In [153]:
df['baseline_review'] = np.nan
for num, row in df.iterrows():
    key = row['id']
    if key in baseline_reviews.keys():
        text = baseline_reviews[key]
        df.loc[num, 'baseline_review'] = text

In [154]:
df['generated_review'] = df['reviews.text'] + '@@@' + df['generated_end']

In [157]:
#df[['id', 'baseline_review', 'generated_review']].to_csv('test_text_evaluation.csv', sep=';', index=False)

In [184]:
with open('all_keywords_with_top20.json', 'r', encoding='utf-8') as f:
    all_keywords = json.loads(f.read())

In [196]:
def keywords_score(keywords, text):
    fulltext = 0
    for keyword in keywords:
        if keyword in text:
            fulltext += 1
    
    keywords_splitted = []
    [keywords_splitted.extend(word_tokenize(keyword)) for keyword in keywords]
    keywords_splitted = set(keywords_splitted)
    splitted = 0
    for keyword in keywords_splitted:
        if keyword in text:
            splitted += 1
    if not len(keywords) == 0:
        fulltext = fulltext / len(keywords)
        splitted = splitted / len(keywords_splitted)
        
    return {
        'fulltext keyword metric': fulltext,
        'splitted keyword metric': splitted
    }

In [197]:
for num, row in tqdm(df.iterrows()):
    key = row['id']
    words = all_keywords[key]
    words = words['pos'] + words['neg']
    text = row['generated_end']
    metrics = keywords_score(words, text)
    df.loc[num, 'fulltext keyword metric'] = metrics['fulltext keyword metric']
    df.loc[num, 'splitted keyword metric'] = metrics['splitted keyword metric']

155it [00:00, 453.51it/s]


In [201]:
df.to_excel('keyword_evaluation.xlsx', index=False)